## Imports

In [79]:
# Setup
import pandas as pd
import neo4j
import sqldf
from graphdatascience import GraphDataScience
from datetime import datetime 

# Viz
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_confusion_matrix


## Setup

In [9]:
host = "bolt://localhost:7687"
user = "neo4j"
password= "letmein"

gds = GraphDataScience(host, auth=(user, password))

print(gds.version())

2.1.11


## Load PPP Data
#### Data from https://www.kaggle.com/datasets/timothyeastridge/paycheck-protection-program

In [12]:
import os
os.chdir("C:\\Users\\TimEa\\OneDrive\\Python\\Scripts\\PPP")
os.listdir(os.curdir)  #files and directories

['.ipynb_checkpoints',
 'pandemic_response_fraud_Neo4j.ipynb',
 'PPP.ipynb',
 'PPP_Dataset.csv',
 'PPP_Dataset_10k_records.csv']

In [28]:
df = pd.read_csv("PPP_Dataset_10k_records.csv")
df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,file_name,Loan_Outstanding_Am,Outstanding_50K
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 HIGHWAY 15 SOUTH,SUMTER,UNKNOWN,29150-9662,12/18/2020,...,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020,public_150k_plus_220703.csv,-4194.59,0
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 SOUTHRAIL ROAD,NORTH CHARLESTON,UNKNOWN,29420-9000,09/28/2021,...,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021,public_150k_plus_220703.csv,-9408.45,0
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,UNKNOWN,98112-2922,03/17/2021,...,CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021,public_150k_plus_220703.csv,-5322.49,0
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,UNKNOWN,92407-1740,10/16/2021,...,CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,395264.11,09/10/2021,public_150k_plus_220703.csv,104606.89,1
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,UNKNOWN,UNKNOWN,UNKNOWN,NaN,08/17/2021,...,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021,public_150k_plus_220703.csv,-3382.35,0


## Explore Data

In [29]:
# Column Names
list(df.columns)

['LoanNumber',
 'DateApproved',
 'SBAOfficeCode',
 'ProcessingMethod',
 'BorrowerName',
 'BorrowerAddress',
 'BorrowerCity',
 'BorrowerState',
 'BorrowerZip',
 'LoanStatusDate',
 'LoanStatus',
 'Term',
 'SBAGuarantyPercentage',
 'InitialApprovalAmount',
 'CurrentApprovalAmount',
 'UndisbursedAmount',
 'FranchiseName',
 'ServicingLenderLocationID',
 'ServicingLenderName',
 'ServicingLenderAddress',
 'ServicingLenderCity',
 'ServicingLenderState',
 'ServicingLenderZip',
 'RuralUrbanIndicator',
 'HubzoneIndicator',
 'LMIIndicator',
 'BusinessAgeDescription',
 'ProjectCity',
 'ProjectCountyName',
 'ProjectState',
 'ProjectZip',
 'CD',
 'JobsReported',
 'NAICSCode',
 'Race',
 'Ethnicity',
 'UTILITIES_PROCEED',
 'PAYROLL_PROCEED',
 'MORTGAGE_INTEREST_PROCEED',
 'RENT_PROCEED',
 'REFINANCE_EIDL_PROCEED',
 'HEALTH_CARE_PROCEED',
 'DEBT_INTEREST_PROCEED',
 'BusinessType',
 'OriginatingLenderLocationID',
 'OriginatingLender',
 'OriginatingLenderCity',
 'OriginatingLenderState',
 'Gender',
 'Vete

In [41]:
import plotly.express as px

fig = px.histogram(df, x="CurrentApprovalAmount",
                   title='Histogram of PPP Loan Amounts',
                   labels={'CurrentApprovalAmount':'Loan Approval Amount'}, # can specify one label per df column
                   opacity=0.8,
                   marginal="rug",
                   color_discrete_sequence=['indianred'] # color of histogram bars
                   )
fig.show()

# Create Unique Loan and Loan Amount Nodes

In [73]:
loan_nodes = sqldf.run("""
SELECT LoanNumber, sum(CurrentApprovalAmount) as LoanAmount
FROM df
GROUP BY LoanNumber
""")
loan_nodes.head()

,LoanNumber,LoanAmount
0,1000627106,497000.0
1,1001567103,746200.0
2,1002397200,232655.1
3,1005178402,196565.0
4,1005927101,224082.5


### Create Constraint

In [44]:
c = '''
// Create constraints
CREATE CONSTRAINT loan IF NOT EXISTS ON (l:LoanNumber) ASSERT l.LoanNumber IS UNIQUE;
'''

gds.run_cypher(c)

""


### Import LoanNumber Nodes

In [49]:
os.chdir("C:\\Users\\TimEa\\AppData\\Local\\Neo4j\\Relate\\Data\\dbmss\\dbms-06964896-0767-4d83-86ea-3c0db0c181e0\\import")
loan_nodes.to_csv("loan_nodes.csv",index=False)

In [83]:
start_time = datetime.now() 
c = '''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "file:///loan_nodes.csv" AS row
MERGE (n:LoanNumber {LoanNumber: toInteger(row.LoanNumber)
, LoanAmount: toInteger(row.LoanAmount)
})
RETURN COUNT(*);
'''

gds.run_cypher(c)

,COUNT(*)
0,10000


In [84]:
# Elapsed Time
time_elapsed = datetime.now() - start_time 

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 0:00:00.949871


# Create Unique Entity Nodes

In [53]:
entity_nodes = sqldf.run("""
SELECT BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState
, sum(CurrentApprovalAmount) as LoanAmount
, max(BusinessAgeDescription) as BusinessAgeDescription
, max(Outstanding_50K) as Outstanding_50K
, count(*) as Entity_Loan_Count
FROM df
GROUP BY 1,2,3,4
""")
entity_nodes.head()

,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,LoanAmount,BusinessAgeDescription,Outstanding_50K,Entity_Loan_Count
0,None,104 MAIN STREET,DILLINGHAM,AK,156767.42,Existing or more than 2 years old,0,1
1,*PARKER BUILDING MAINTENANCE INC,1233 LIBERTY PARK LOOP,VESTAVIA,AL,235000.00,Existing or more than 2 years old,0,1
2,1 SOURCE LLC,3240 MOFFETT RD,MOBILE,AL,218940.00,Existing or more than 2 years old,0,1
3,10 BELOW LLC,2500 CRESTWOOD BLVD 500,IRONDALE,AL,1393000.00,Existing or more than 2 years old,0,1
4,2215 NORTHPORT OPCO LLC,2215 32ND STREET,NORTHPORT,AL,1358800.00,New Business or 2 years or less,1,1


### Create Indexes

In [55]:
c = '''
CREATE INDEX ON :Entity(BorrowerName)'''
gds.run_cypher(c)
c = '''
CREATE INDEX ON :Entity(BorrowerAddress)'''
gds.run_cypher(c)
c = '''
CREATE INDEX ON :Entity(BorrowerCity)'''
gds.run_cypher(c)
c = '''
CREATE INDEX ON :Entity(BorrowerState)'''
gds.run_cypher(c)

""


### Import Entity Nodes

In [56]:
os.chdir("C:\\Users\\TimEa\\AppData\\Local\\Neo4j\\Relate\\Data\\dbmss\\dbms-06964896-0767-4d83-86ea-3c0db0c181e0\\import")
entity_nodes.to_csv("entity_nodes.csv",index=False)

In [58]:
c = '''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "file:///entity_nodes.csv" AS row
MERGE (n:Entity {BorrowerName: COALESCE(row.BorrowerName,"")
, BorrowerAddress: row.BorrowerAddress
, BorrowerCity: row.BorrowerCity
, BorrowerState: row.BorrowerState
})
RETURN COUNT(*);
'''

gds.run_cypher(c)

,COUNT(*)
0,9671


# Create Edges / Relationships

In [60]:
edges_borrower_to_loan = sqldf.run("""
SELECT LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState
FROM df
""")
edges_borrower_to_loan.head()

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState
0,9547507704,"SUMTER COATINGS, INC.",2410 HIGHWAY 15 SOUTH,SUMTER,UNKNOWN
1,9777677704,"PLEASANT PLACES, INC.",7684 SOUTHRAIL ROAD,NORTH CHARLESTON,UNKNOWN
2,5791407702,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,UNKNOWN
3,6223567700,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,UNKNOWN
4,9662437702,AERO BOX LLC,UNKNOWN,UNKNOWN,UNKNOWN


### Import Relationships

In [61]:
os.chdir("C:\\Users\\TimEa\\AppData\\Local\\Neo4j\\Relate\\Data\\dbmss\\dbms-06964896-0767-4d83-86ea-3c0db0c181e0\\import")
edges_borrower_to_loan.to_csv("edges_borrower_to_loan.csv",index=False)

In [64]:
c = '''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "file:///edges_borrower_to_loan.csv" AS row
MATCH (source:Entity {BorrowerName: COALESCE(row.BorrowerName,""), BorrowerAddress: row.BorrowerAddress, BorrowerCity: row.BorrowerCity, BorrowerState: row.BorrowerState})
MATCH (target:LoanNumber {LoanNumber: toInteger(row.LoanNumber)})
MERGE (source)-[:RECEIVED_LOAN]->(target)
RETURN COUNT(*);
'''

gds.run_cypher(c)

,COUNT(*)
0,10000


### Connect Entities with Shared Address

In [65]:
c = '''
MATCH (a:Entity),(b:Entity)
WHERE a.BorrowerAddress = b.BorrowerAddress
    AND a.BorrowerCity = b.BorrowerCity
    AND a.BorrowerState = b.BorrowerState
    AND id(a) < id(b)
CREATE (a)-[:SHARES_ADDRESS]->(b) 
RETURN COUNT(*);
'''

gds.run_cypher(c)

,COUNT(*)
0,1215


### Connect Entities with Shared Address

In [65]:
c = '''
MATCH (a:Entity),(b:Entity)
WHERE a.BorrowerAddress = b.BorrowerAddress
    AND a.BorrowerCity = b.BorrowerCity
    AND a.BorrowerState = b.BorrowerState
    AND id(a) < id(b)
CREATE (a)-[:SHARES_ADDRESS]->(b) 
RETURN COUNT(*);
'''

gds.run_cypher(c)

,COUNT(*)
0,1215


### Which Entities Received the Highest Number of Loans?

In [67]:
c = '''
MATCH (a:Entity)-[r1:RECEIVED_LOAN]->(b:LoanNumber)
RETURN a.BorrowerName as BorrowerName, sum(b.LoanAmount) as LoanAmount, count(r1) as LoanCount
ORDER BY LoanCount * log(LoanAmount) DESC
LIMIT 10
'''

gds.run_cypher(c)

,BorrowerName,LoanAmount,LoanCount
0,YOUNG MEN'S CHRISTIAN ASSOCIATION,4038132,3
1,ASBURY UNITED METHODIST CHURCH,1345409,3
2,FIRST UNITED METHODIST CHURCH,905222,3
3,ACTION ENTERPRISE HOLDINGS LLC,12000000,2
4,PCH HOTELS AND RESORTS INC,11501800,2
5,BIRMINGHAM-SOUTHERN COLLEGE,6407500,2
6,ADTRAV CORPORATION,6023032,2
7,VULCAN INC.,5834600,2
8,RUSSO CORPORATION,5705981,2
9,THE MCPHERSON COMPANIES INC,5406100,2


### View Info for Loan Recipient 

In [71]:
c = '''
MATCH (a:Entity)
WHERE a.BorrowerName = 'ASBURY UNITED METHODIST CHURCH'
RETURN a.BorrowerName, a.BorrowerAddress, a.BorrowerCity, a.BorrowerState
LIMIT 10
'''

gds.run_cypher(c)

,a.BorrowerName,a.BorrowerAddress,a.BorrowerCity,a.BorrowerState
0,ASBURY UNITED METHODIST CHURCH,6690 CAHABA VALLEY RD,BIRMINGHAM,AL
1,ASBURY UNITED METHODIST CHURCH,980 HUGHES RD,MADISON,AL
